<a href="https://colab.research.google.com/github/duneag2/intro-dl/blob/main/DLMATHon/uh/dlmathon2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install pyheif

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 64.0 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
! unzip -qq '/content/drive/MyDrive/ddubuk_cls.zip' -d '/content/drive/MyDrive'

replace /content/drive/MyDrive/ddubuk/test/ddubuk/IMG_0974.HEIC? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace /content/drive/MyDrive/ddubuk/test/ddubuk/IMG_1847.HEIC? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace /content/drive/MyDrive/ddubuk/test/ddubuk/IMG_1848.HEIC? [y]es, [n]o, [A]ll, [N]one, [r]ename: r
new name: 

In [3]:
%cd /content/drive/MyDrive/ddubuk/

/content/drive/MyDrive/ddubuk


In [9]:
import os
import shutil
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.utils.data as data
from torchvision import datasets, models, transforms
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
import random
import pyheif

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using PyTorch version:', torch.__version__, ' Device:', device)

Using PyTorch version: 2.3.1+cu121  Device: cpu


In [10]:
# Define paths
train_dir = '/content/drive/MyDrive/ddubuk/train'
test_dir = '/content/drive/MyDrive/ddubuk/test'

# Convert images to a common format (JPEG)
def convert_images_to_jpg(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            filepath = os.path.join(root, file)
            new_filepath = os.path.splitext(filepath)[0] + '.jpg'

            if file.lower().endswith('.heic'):
                try:
                    heif_file = pyheif.read(filepath)
                    image = Image.frombytes(
                        heif_file.mode,
                        heif_file.size,
                        heif_file.data,
                        "raw",
                        heif_file.mode,
                        heif_file.stride,
                    )
                    image = image.convert('RGB')
                    image.save(new_filepath, "JPEG")
                    os.remove(filepath)
                except Exception as e:
                    print(f"Could not convert {filepath}: {e}")

            elif not file.lower().endswith('.jpg'):
                try:
                    img = Image.open(filepath)
                    img = img.convert('RGB')
                    img.save(new_filepath, 'JPEG')
                    os.remove(filepath)
                except Exception as e:
                    print(f"Could not convert {filepath}: {e}")

convert_images_to_jpg(train_dir)
convert_images_to_jpg(test_dir)

In [11]:
# Data augmentation and normalization for training
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [12]:
# Load data from folders
train_dataset = datasets.ImageFolder(train_dir, transform=data_transforms['train'])
test_dataset = datasets.ImageFolder(test_dir, transform=data_transforms['test'])

# Split the train_dataset into train and validation datasets
train_indices, val_indices = train_test_split(list(range(len(train_dataset))), test_size=0.2, stratify=train_dataset.targets)

train_set = Subset(train_dataset, train_indices)
val_set = Subset(train_dataset, val_indices)

# Create data loaders
batch_size = 32
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [15]:
# Load pre-trained ResNet50 model
model_ft = models.resnet50(pretrained=True)

# Modify the final fully connected layer to match the number of classes (ddubuk and not_ddubuk)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)  # 2 classes: ddubuk and not_ddubuk

model_ft = model_ft.to(device)

In [16]:
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
                dataloader = train_loader
            else:
                model.eval()   # Set model to evaluate mode
                dataloader = val_loader

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward
                # Track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / len(dataloader.dataset)
            epoch_acc = running_corrects.double() / len(dataloader.dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

        print()

    print(f'Best val Acc: {best_acc:4f}')

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model

# Train the model
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=25)

Epoch 0/24
----------


In [ ]:
def test_model(model, test_loader):
    model.eval()
    running_corrects = 0

    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        with torch.no_grad():
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

        running_corrects += torch.sum(preds == labels.data)

    test_acc = running_corrects.double() / len(test_loader.dataset)
    print(f'Test Accuracy: {test_acc:.4f}')

# Test the model
test_model(model_ft, test_loader)

In [ ]:
torch.save(model_ft.state_dict(), '/content/drive/MyDrive/ddubuk/model_resnet50.pth')